### Modelado

En este notebook utilizaremos los conjuntos que hemos inspeccionado, adecuado y preprocesado para entrenar modelos que nos ayuden a analizar los mensajes de texto y detectar si se trata de **spam** o no.

El siguiente script está dividido en los siguientes bloques:

- **BLOQUE A**: carga de datos preprocesados.
- **BLOQUE B**: entrenamiento y inferencia con distintos modelos de ML.
- **BLOQUE C**: entrenamiento y inferencia con una red neuronal (librería Keras).
- **BLOQUE D**: transfer learning con un modelo preentrenado.

In [2]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report


from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer

2024-11-26 16:17:37.410259: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 16:17:38.192827: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-26 16:17:38.192979: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-26 16:17:38.197821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 16:17:38.570266: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 16:17:38.571971: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### BLOQUE A: Carga de datos
Antes de comenzar, cargaremos los datos que han sido adecuados en nuestra fase anterior de preprocesado

In [3]:
# Carga textos vectorizados
with open('../data/x_train_vec.pkl', 'rb') as f:
    X_train_vec = pickle.load(f)

with open('../data/x_test_vec.pkl', 'rb') as f:
    X_test_vec = pickle.load(f)

In [4]:
# Carga conjuntos de las etiquetas
with open('../data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('../data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### BLOQUE B: Entrenamiento de distinto modelos de ML

#### Regresión logistica

La regresión logística es un modelo estadístico para estudiar las relaciones entre un conjunto de variables cualitativas Xi y una variable cualitativa Y. Utilizando la función sigmoide, asigna valores entre 0 y 1, facilitando la predicción de categorías, como positivo o negativos, spam o no spam, en aplicaciones prácticas.

Más información acerca de este modelo [aquí](https://cienciadedatos.net/documentos/py17-regresion-logistica-python.html).

In [5]:
# Creamos el modelo
log_model = LogisticRegression()

In [6]:
# Entrenamiento o ajuste del modelo con los datos de entrenamiento
log_model.fit(X_train_vec, y_train)

LogisticRegression()

In [7]:
# Predecimos sobre los datos de entrenamiento
y_pred_train = log_model.predict(X_train_vec)

# Mostramos el "classification report"
print('Resultados conjunto de entrenamiento:\n')
print(classification_report(y_train, y_pred_train))

Resultados conjunto de entrenamiento:

              precision    recall  f1-score   support

         ham       0.96      0.99      0.97       590
        spam       0.99      0.96      0.97       590

    accuracy                           0.97      1180
   macro avg       0.97      0.97      0.97      1180
weighted avg       0.97      0.97      0.97      1180



In [8]:
# Predecimos sobre los datos de test
y_pred_test = log_model.predict(X_test_vec)

# Mostramos el "classification report" y "accuracy"
accuracy = accuracy_score(y_test, y_pred_test)
print('Resultados conjunto de test:\n')
print(f'Accuracy: {accuracy:.2f}\n')
print(classification_report(y_test, y_pred_test))

Resultados conjunto de test:

Accuracy: 0.97

              precision    recall  f1-score   support

         ham       0.98      0.98      0.98       659
        spam       0.92      0.93      0.92       147

    accuracy                           0.97       806
   macro avg       0.95      0.95      0.95       806
weighted avg       0.97      0.97      0.97       806



#### Gradient boosting

¿Qué es Boosting?

Boosting es un meta-algoritmo de aprendizaje automático que reduce el sesgo y la varianza en un contexto de aprendizaje supervisado. Consiste en combinar los resultados de varios clasificadores débiles para obtener un clasificador robusto. Cuando se añaden estos clasificadores débiles, se hace de modo que éstos tengan diferente peso en función de la exactitud de sus predicciones. Tras añadir un clasificador débil, los datos cambian su estructura de pesos: los casos mal clasificados ganan peso y los que son clasificados correctamente pierden peso.

**Gradient Boosting (GB)** o Potenciación del gradiente consiste en plantear el problema como una optimización numérica en el que el objetivo es minimizar una función de coste añadiendo clasificadores débiles mediante el descenso del gradiente. Involucra tres elementos:

- La **función de coste** a optimizar: depende del tipo de problema a resolver.
- Un **clasificador débil** para hacer las predicciones: por lo general se usan árboles de decisión.
- Un **modelo que añade (ensambla) los clasificadores débiles para minimizar la función de coste**: se usa el descenso del gradiente para minimizar el coste al añadir árboles.

Los hiperparámetros más importantes que intervienen en este algoritmo (aunque no todos) son:

- **learning_rate**: determina el impacto de cada árbol en la salida final. Se parte de una estimación inicial que se va actualizando con la salida de cada árbol. Es el parámetro que controla la magnitud de las actualizaciones.
- **n_estimators**: número de clasificadores débiles a utilizar.

Como en este caso utilizaremos árboles de decisión como clasificadores débiles a ensamblar, también debemos tener en cuenta los hiperparámetros que afectan a esta clase de modelos. En este caso:

- **max_depth**: profundidad máxima del árbol.

Más información sobre el modelo que se utiliza en este ejemplo y de sus parámetros [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html).

In [9]:
# Creamos el modelo introduciendo los valores de los parámetros:
gb_clf = GradientBoostingClassifier(n_estimators=150, learning_rate=0.2, max_depth=3, random_state=0)

In [10]:
# Entrenamiento o ajuste del modelo con los datos de entrenamiento
gb_clf.fit(X_train_vec, y_train)

GradientBoostingClassifier(learning_rate=0.2, n_estimators=150, random_state=0)

In [11]:
# Predecimos sobre los datos de entrenamiento
pred_train = gb_clf.predict(X_train_vec)

# Mostramos el "classification report"
print('Resultados conjunto de entrenamiento:\n')
print(classification_report(y_train, pred_train))

Resultados conjunto de entrenamiento:

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00       590
        spam       1.00      1.00      1.00       590

    accuracy                           1.00      1180
   macro avg       1.00      1.00      1.00      1180
weighted avg       1.00      1.00      1.00      1180



In [12]:
# Predecimos sobre los datos de test
pred_test = gb_clf.predict(X_test_vec)

# Mostramos el "classification report" y "accuracy"
accuracy = accuracy_score(y_test, pred_test)
print('Resultados conjunto de test:\n')
print(f'Accuracy: {accuracy:.2f}\n')
print(classification_report(y_test, pred_test))

Resultados conjunto de test:

Accuracy: 0.95

              precision    recall  f1-score   support

         ham       0.98      0.95      0.97       659
        spam       0.80      0.93      0.86       147

    accuracy                           0.95       806
   macro avg       0.89      0.94      0.91       806
weighted avg       0.95      0.95      0.95       806



### BLOQUE C: entrenamiento y inferencia con una red neuronal (librería Keras)

Las redes neuronales artificiales son una rama de la Inteligencia Artificial que se basa en la simulación de la estructura y funcionamiento del cerebro humano para procesar información.

Consisten en una serie de nodos interconectados que reciben información, la procesan y producen una salida. Dichos nodos, conocidos como neuronas artificiales, pueden ser ajustados para optimizar la salida de la red, lo que permite que la red aprenda y se adapte a diferentes tipos de entradas.

*Curiosidad: cuando una red neuronal tiene simplemente más de tres capas, nace el conocido como **Deep Learning** o aprendizaje profundo.*

Más información sobre este tipo de modelos [aquí](https://openwebinars.net/blog/que-son-las-redes-neuronales-y-sus-aplicaciones/).

En este caso, entrenaremos la red para que también obtenga una representación de los textos analizados de manera directa, sin necesidad de utilizar el modelo de vectorizado TF-IDF antes calculado.

In [13]:
# Cargar el dataset (ajusta la ruta según el archivo que tengas)
df = pd.read_csv('../data/spam_filtered.csv')

# Verifica el dataset
df.head()

,target,text,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,ham,U dun say so early hor... U c already then say...,49
3,ham,"Nah I don't think he goes to usf, he lives aro...",61
4,spam,FreeMsg Hey there darling it's been 3 week's n...,148


In [14]:
# Preprocesamiento del texto
X = df['text']  # Supón que la columna 'text' contiene los mensajes
y = df['target']  # 'label' debe ser spam/no spam

In [15]:
# Convertir las etiquetas 'spam'/'ham' a números
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [16]:
# ¿Que valores asume la variale target transformada? 
y

array([0, 1, 0, ..., 0, 0, 0])

In [17]:
# ¿A qué corresponden los nuevos valores?
df['target'].values

array(['ham', 'spam', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [18]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [19]:
# Tokenización y padding para convertir el texto en secuencias de enteros
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [20]:
# ¿Como son transformados los textos con Tokenizer?
X_train_seq[0]

[5,
 370,
 813,
 5,
 746,
 69,
 2401,
 321,
 88,
 549,
 300,
 69,
 2402,
 2403,
 2404,
 321,
 99,
 814,
 2405,
 1893,
 196,
 116,
 370,
 815,
 18,
 2406,
 116,
 746,
 2407,
 65,
 5,
 2408,
 57,
 2409,
 1179,
 116,
 746,
 1894,
 16,
 57,
 76,
 18,
 691,
 116,
 371,
 116,
 370,
 1540,
 56,
 1180,
 80,
 1541,
 99,
 88,
 494,
 2410,
 1895,
 7,
 495,
 26,
 692,
 883,
 116,
 370,
 1333,
 1334,
 9,
 116,
 693,
 14,
 11,
 816,
 80,
 1896,
 116,
 371,
 1897,
 85,
 816,
 80,
 2411,
 2412,
 2413,
 2414,
 1898,
 1334,
 2415,
 1181,
 14,
 116,
 694,
 3,
 1335,
 47,
 695,
 116,
 306,
 817]

In [21]:
# Padding para asegurar que todas las secuencias tengan la misma longitud
X_train_pad = pad_sequences(X_train_seq, padding='post', maxlen=100)
X_test_pad = pad_sequences(X_test_seq, padding='post', maxlen=100)

Estructura de la red neuronal:
1. Capa de Embedding: 
    - input_dim=10000: Esto se refiere al tamaño del vocabulario (número total de palabras). 

    - output_dim=64: Este especifica la dimensión de los embeddings. En lugar de representar cada palabra con un valor binario o en un vector disperso (como ocurre con TF-IDF), las palabras se representarán por un vector denso de 64 dimensiones. 

    - input_length=100: se define la longitud máxima de las secuencias de entrada. Cada entrada de texto se convierte en una secuencia de índices de palabras (tokens), y si la longitud de una secuencia es menor que 100, se rellenará con ceros. Si es mayor, se truncará.
2. Capa de Pooling: Esta capa realiza un *pooling global promedio* sobre las secuencias de embeddings. Dado que las entradas son secuencias de longitud 100, esta capa tomará el promedio de los vectores de embeddings de las 100 palabras (tokens) en la secuencia, lo que reduce la representación de la secuencia de una matriz de 100x64 a un solo vector de 64 dimensiones

3. Capa densa (Fully connected): Esta capa toma la salida del pooling (un vector de 64 dimensiones) y lo pasa a través de una **capa densa con 64 neuronas** y la **función de activación ReLU**. Esto ayuda a la red a aprender características más complejas del texto procesado.

4. Capa de salida: esta capa tiene una sola neurona con la **función de activación Sigmoid**, que es adecuada para clasificación binaria (en este caso, para clasificar como spam o no spam). La salida de esta capa será un valor entre 0 y 1, que puedes interpretar como la probabilidad de que un mensaje sea spam.



In [22]:
# Modelo simple de red neuronal para clasificación de texto
nn_simple = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=100),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Para clasificación binaria (spam/no spam)
])

In [23]:
# Compilar el modelo
nn_simple.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
nn_simple.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

Epoch 1/10
101/101 [==============================] - 2s 12ms/step - loss: 0.5151 - accuracy: 0.8091 - val_loss: 0.4283 - val_accuracy: 0.8176
Epoch 2/10
101/101 [==============================] - 1s 13ms/step - loss: 0.3854 - accuracy: 0.8169 - val_loss: 0.3206 - val_accuracy: 0.8176
Epoch 3/10
101/101 [==============================] - 1s 12ms/step - loss: 0.2252 - accuracy: 0.8973 - val_loss: 0.1493 - val_accuracy: 0.9615
Epoch 4/10
101/101 [==============================] - 1s 12ms/step - loss: 0.0913 - accuracy: 0.9761 - val_loss: 0.0771 - val_accuracy: 0.9702
Epoch 5/10
101/101 [==============================] - 1s 11ms/step - loss: 0.0493 - accuracy: 0.9854 - val_loss: 0.0638 - val_accuracy: 0.9826
Epoch 6/10
101/101 [==============================] - 1s 12ms/step - loss: 0.0338 - accuracy: 0.9901 - val_loss: 0.0598 - val_accuracy: 0.9826
Epoch 7/10
101/101 [==============================] - 1s 11ms/step - loss: 0.0251 - accuracy: 0.9929 - val_loss: 0.0557 - val_accuracy: 0.9814

In [24]:
# Predecimos sobre los datos de test
pred_test = nn_simple.predict(X_test_pad)

26/26 [==============================] - 0s 1ms/step


In [25]:
# ¿Qué contiene pred_test? ¿Qué son?
pred_test

array([[1.08566892e-04],
       [9.99488473e-01],
       [9.99765038e-01],
       [8.28603879e-05],
       [1.38014893e-03],
       [2.03493983e-03],
       [9.66416788e-04],
       [3.72315873e-03],
       [9.99837875e-01],
       [1.19081314e-03],
       [3.96732660e-03],
       [7.28066802e-01],
       [9.99416649e-01],
       [7.30753964e-05],
       [5.66092458e-05],
       [8.50686152e-03],
       [4.00147750e-04],
       [4.45264959e-05],
       [9.96309876e-01],
       [3.33562421e-05],
       [1.72596285e-03],
       [4.91901301e-03],
       [3.96300061e-03],
       [9.99557078e-01],
       [1.81678508e-03],
       [2.83042667e-03],
       [3.98756452e-02],
       [9.99849498e-01],
       [1.05822897e-02],
       [4.52001055e-04],
       [2.36934167e-04],
       [5.02330018e-04],
       [9.42510545e-01],
       [9.51429248e-01],
       [9.97791409e-01],
       [9.99255657e-01],
       [2.24135388e-02],
       [1.64665151e-02],
       [2.81183515e-04],
       [1.26757503e-01],


In [26]:
# Convertir las predicciones a clases (spam o no spam)
# La salida será un valor entre 0 y 1 (probabilidad de spam)
predicted_labels = (pred_test > 0.5).astype(int)  # Clasifica como spam (1) si la probabilidad > 0.5, sino no spam (0)
predicted_labels

array([[0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
    

In [27]:
# Mostramos el "classification report" y "accuracy"
accuracy = accuracy_score(y_test, predicted_labels)
print('Resultados conjunto de test:\n')
print(f'Accuracy: {accuracy:.2f}\n')
print(classification_report(y_test, predicted_labels))

Resultados conjunto de test:

Accuracy: 0.98

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       659
           1       0.97      0.93      0.95       147

    accuracy                           0.98       806
   macro avg       0.98      0.96      0.97       806
weighted avg       0.98      0.98      0.98       806



### Transfer Learning con un modelo preentrenado

**BERT** (Bidirectional Encoder Representations from Transformers) es una técnica basada en redes neuronales para el pre-entrenamiento del procesamiento del lenguaje natural (NLP) desarrollada por Google. Se trata de un codificador que obtiene representaciones bidireccionales, lo que significa que puede entender el significado de una palabra en relación con las palabras que la rodean. Esto permite al modelo aprender a **interpretar el lenguaje en general y ser utilizado como modelo base**, de manera que, en caso de querer especializarlo en una tarea en particular (detección de spams, por ejemplo), solo es necesario añadir unas capas adicionales al mismo.

El modelo original de BERT se entrenó utilizando dos grandes conjuntos de datos en lengua inglesa: BookCorpus y Wikipedia en inglés. 

*Curiosidad: actualmente, Google utiliza BERT en su motor de búsqueda para perfeccionar la comprensión de las búsquedas de los usuarios.*

Por su parte, **DistilBERT** (Distilled version of BERT) es una versión más ligera, rápida y menos costosa del modelo BERT, como su propio nombre indica.

Más información acerca de BERT [aquí](https://huggingface.co/blog/bert-101), y de DistilBERT [aquí](https://huggingface.co/docs/transformers/model_doc/distilbert) y [aquí](https://medium.com/huggingface/distilbert-8cf3380435b5).

In [28]:
# Cargar el tokenizer de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/home/cristian.robledo/anaconda3/envs/crypto_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Tokenización de los textos
X_train_enc = tokenizer(list(X_train[:100]), padding=True, truncation=True, max_length=100, return_tensors='tf') # Usamos 100 observaciones para poder ejecutarlo en local
X_test_enc = tokenizer(list(X_test), padding=True, truncation=True, max_length=100, return_tensors='tf')

# Cargar el modelo preentrenado DistilBERT para clasificación de texto
model_bert = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Compilar el modelo
model_bert.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento con transfer learning
model_bert.fit(X_train_enc['input_ids'], y_train[:100], epochs=3, batch_size=128, validation_data=(X_test_enc['input_ids'], y_test))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
1/1 [==============================] - 93s 93s/step - loss: 0.9321 - accuracy: 0.7400 - val_loss: 0.7608 - val_accuracy: 0.8176
Epoch 2/3
1/1 [==============================] - 80s 80s/step - loss: 1.4566 - accuracy: 0.8200 - val_loss: 1.0840 - val_accuracy: 0.8176
Epoch 3/3
1/1 [==============================] - 91s 91s/step - loss: 0.9741 - accuracy: 0.8200 - val_loss: 0.4520 - val_accuracy: 0.8176


In [38]:
# Evaluamos sobre el conjunto de test y vemos métrica
loss, accuracy = model_bert.evaluate(X_test_enc['input_ids'], y_test)
print(f"Accuracy DistilBERT model: {accuracy}")

26/26 [==============================] - 64s 2s/step - loss: 0.4520 - accuracy: 0.8176
Accuracy DistilBERT model: 0.8176178932189941
